# Instacart Data Generation for CDC Pipeline

This notebook loads Instacart data into PostgreSQL to simulate an OLTP application:
- **Dimension tables** (aisles, departments, products): Load all data at once
- **Transactional tables** (orders, order_products): Load incrementally to simulate real-time data generation

## 1. Import Required Libraries

In [2]:
import psycopg2
from psycopg2.extras import execute_batch
import pandas as pd
import time
from datetime import datetime
import os
import random
import json
import logging

from dotenv import load_dotenv
load_dotenv("../.env")

True

## 2. Database Configuration

In [3]:
# Database connection parameters
DB_CONFIG = {
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'database': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD')
}

# Data directory
DATA_DIR = '../data'

# Progress log file
PROGRESS_LOG_FILE = '../data/loading_progress.json'

# Application log file
LOG_FILE = '../data/data_loading.log'

# Batch configuration for incremental loading
MIN_ORDERS_PER_BATCH = 50  # Minimum number of orders per batch
MAX_ORDERS_PER_BATCH = 150  # Maximum number of orders per batch
MIN_SLEEP_SECONDS = 1  # Minimum seconds to sleep between batches
MAX_SLEEP_SECONDS = 10  # Maximum seconds to sleep between batches

## 3. Helper Functions

In [4]:
def setup_logging():
    """Configure logging to file and console"""
    # Create logs directory if it doesn't exist
    os.makedirs(os.path.dirname(LOG_FILE), exist_ok=True)
    
    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(LOG_FILE, mode='a'),
            logging.StreamHandler()  # Also log to console for monitoring
        ]
    )
    return logging.getLogger(__name__)

# Initialize logger
logger = setup_logging()

def get_db_connection():
    """Create and return a database connection"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        return conn
    except Exception as e:
        logger.error(f"Error connecting to database: {e}")
        raise

def load_csv(filename):
    """Load CSV file into a pandas DataFrame"""
    filepath = os.path.join(DATA_DIR, filename)
    logger.info(f"Loading {filename}...")
    df = pd.read_csv(filepath)
    logger.info(f"Loaded {len(df)} rows from {filename}")
    return df

def convert_to_native_types(df):
    """Convert numpy dtypes to native Python types"""
    df = df.copy()
    for col in df.columns:
        if df[col].dtype.name.startswith('int'):
            df[col] = df[col].astype('Int64')  # Nullable integer type
        elif df[col].dtype.name.startswith('float'):
            df[col] = df[col].astype('float64')
    return df

def insert_dataframe_batch(conn, df, table_name, columns):
    """Insert DataFrame into database table using batch insert"""
    cursor = conn.cursor()
    
    # Convert to native types
    df = convert_to_native_types(df)
    
    # Prepare the insert query
    placeholders = ','.join(['%s'] * len(columns))
    insert_query = f"INSERT INTO {table_name} ({','.join(columns)}) VALUES ({placeholders})"
    
    # Convert DataFrame to list of tuples with proper None handling
    data = []
    for _, row in df[columns].iterrows():
        tuple_data = tuple(None if pd.isna(val) else int(val) if isinstance(val, (pd.Int64Dtype, int)) and not isinstance(val, bool) 
                          else float(val) if isinstance(val, float) else val 
                          for val in row)
        data.append(tuple_data)
    
    # Execute batch insert
    execute_batch(cursor, insert_query, data, page_size=1000)
    conn.commit()
    cursor.close()
    logger.info(f"Inserted {len(data)} rows into {table_name}")

def load_progress():
    """Load progress from log file"""
    if os.path.exists(PROGRESS_LOG_FILE):
        with open(PROGRESS_LOG_FILE, 'r') as f:
            return json.load(f)
    return {'last_order_index': 0, 'total_orders_loaded': 0, 'total_order_products_loaded': 0}

def save_progress(progress):
    """Save progress to log file"""
    os.makedirs(os.path.dirname(PROGRESS_LOG_FILE), exist_ok=True)
    with open(PROGRESS_LOG_FILE, 'w') as f:
        json.dump(progress, f, indent=2)

def insert_batch_with_products(conn, orders_batch, order_products_df):
    """Insert a batch of orders along with their order products"""
    cursor = conn.cursor()
    
    try:
        # Insert orders
        order_columns = ['order_id', 'user_id', 'order_number', 
                        'order_dow', 'order_hour_of_day', 'days_since_prior_order']
        
        # Convert orders batch to native types and prepare data
        orders_batch_native = convert_to_native_types(orders_batch)
        order_data = []
        for _, row in orders_batch_native[order_columns].iterrows():
            tuple_data = tuple(None if pd.isna(val) else int(val) if pd.notna(val) and isinstance(val, (int, float)) and not isinstance(val, bool)
                              else val for val in row)
            order_data.append(tuple_data)
        
        placeholders = ','.join(['%s'] * len(order_columns))
        insert_query = f"INSERT INTO instacart.orders ({','.join(order_columns)}) VALUES ({placeholders})"
        execute_batch(cursor, insert_query, order_data, page_size=len(order_data))
        
        # Get order products for this batch - convert order_ids to Python int
        order_ids = [int(x) for x in orders_batch['order_id'].tolist()]
        batch_order_products = order_products_df[order_products_df['order_id'].isin(order_ids)]
        
        if len(batch_order_products) > 0:
            # Insert order products
            order_product_columns = ['order_id', 'product_id', 'add_to_cart_order', 'reordered']
            
            # Convert to native types and prepare data
            batch_order_products_native = convert_to_native_types(batch_order_products)
            product_data = []
            for _, row in batch_order_products_native[order_product_columns].iterrows():
                tuple_data = tuple(int(val) if pd.notna(val) else None for val in row)
                product_data.append(tuple_data)
            
            placeholders = ','.join(['%s'] * len(order_product_columns))
            insert_query = f"INSERT INTO instacart.order_products ({','.join(order_product_columns)}) VALUES ({placeholders})"
            execute_batch(cursor, insert_query, product_data, page_size=len(product_data))
        
        conn.commit()
        cursor.close()
        
        return len(order_data), len(batch_order_products)
        
    except Exception as e:
        conn.rollback()
        cursor.close()
        raise e

def reset_progress():
    """Reset the progress log to start from the beginning"""
    if os.path.exists(PROGRESS_LOG_FILE):
        os.remove(PROGRESS_LOG_FILE)
        logger.info("Progress log reset. Next run will start from the beginning.")
    else:
        logger.info("No progress log found.")

## 4. Load Dimension Data (All at Once)

Dimension tables are loaded completely as they represent reference data that doesn't change frequently.

In [10]:
def load_dimension_tables():
    """Load all dimension tables (aisles, departments, products)"""
    conn = get_db_connection()
    
    try:
        logger.info("="*60)
        logger.info("LOADING DIMENSION TABLES")
        logger.info("="*60)
        
        # Load aisles
        logger.info("Loading aisles...")
        aisles_df = load_csv('aisles.csv')
        insert_dataframe_batch(conn, aisles_df, 'instacart.aisles', ['aisle_id', 'aisle'])
        
        # Load departments
        logger.info("Loading departments...")
        departments_df = load_csv('departments.csv')
        insert_dataframe_batch(conn, departments_df, 'instacart.departments', ['department_id', 'department'])
        
        # Load products
        logger.info("Loading products...")
        products_df = load_csv('products.csv')
        insert_dataframe_batch(conn, products_df, 'instacart.products', 
                              ['product_id', 'product_name', 'aisle_id', 'department_id'])
        
        logger.info("="*60)
        logger.info("DIMENSION TABLES LOADED SUCCESSFULLY")
        logger.info("="*60)
        
    except Exception as e:
        logger.error(f"Error loading dimension tables: {e}")
        conn.rollback()
        raise
    finally:
        conn.close()

# Execute dimension table loading
load_dimension_tables()


LOADING DIMENSION TABLES

Loading aisles...
Loading aisles.csv...
Loaded 134 rows from aisles.csv
Inserted 134 rows into instacart.aisles

Loading departments...
Loading departments.csv...
Loaded 21 rows from departments.csv
Inserted 21 rows into instacart.departments

Loading products...
Loading products.csv...
Loaded 49688 rows from products.csv
Inserted 49688 rows into instacart.products

DIMENSION TABLES LOADED SUCCESSFULLY



## 5. Load Train Orders Data (Incrementally)

Train orders and order products are loaded incrementally to simulate real-time OLTP application behavior for CDC pipeline testing. Test data is excluded.

In [8]:
def load_train_orders_incrementally():
    """Load train orders and order products incrementally to simulate OLTP behavior"""
    conn = get_db_connection()
    
    try:
        logger.info("="*60)
        logger.info("LOADING TRAIN ORDERS DATA (INCREMENTALLY)")
        logger.info("="*60)
        
        # Load progress
        progress = load_progress()
        start_index = progress['last_order_index']
        
        if start_index > 0:
            logger.info(f"Resuming from order index {start_index}")
            logger.info(f"Previously loaded: {progress['total_orders_loaded']} orders, "
                       f"{progress['total_order_products_loaded']} order products")
        
        # Load orders
        logger.info("Loading orders.csv...")
        orders_df = pd.read_csv(os.path.join(DATA_DIR, 'orders.csv'))
        
        # Filter only train orders
        train_orders = orders_df[orders_df['eval_set'] == 'train'].sort_values('order_id').reset_index(drop=True)
        logger.info(f"Found {len(train_orders)} train orders")
        
        # Load order products
        logger.info("Loading order_products__train.csv...")
        train_products_df = pd.read_csv(os.path.join(DATA_DIR, 'order_products__train.csv'))
        logger.info(f"Found {len(train_products_df)} train order products")
        
        # Process in batches with random sizes
        current_index = start_index
        total_orders = len(train_orders)
        
        try:
            while current_index < total_orders:
                # Random batch size
                batch_size = random.randint(MIN_ORDERS_PER_BATCH, MAX_ORDERS_PER_BATCH)
                end_index = min(current_index + batch_size, total_orders)
                
                # Get batch of orders
                orders_batch = train_orders.iloc[current_index:end_index]
                
                # Insert orders and their products
                orders_inserted, products_inserted = insert_batch_with_products(
                    conn, orders_batch, train_products_df
                )
                
                # Update progress
                current_index = end_index
                progress['last_order_index'] = current_index
                progress['total_orders_loaded'] += orders_inserted
                progress['total_order_products_loaded'] += products_inserted
                save_progress(progress)
                
                # Log progress
                logger.info(f"Train batch completed: {orders_inserted} orders, {products_inserted} order products | "
                           f"Progress: {current_index}/{total_orders} orders ({current_index/total_orders*100:.1f}%)")
                
                # Sleep with random duration to simulate real-time data generation
                if current_index < total_orders:
                    sleep_time = random.randint(MIN_SLEEP_SECONDS, MAX_SLEEP_SECONDS)
                    logger.info(f"Sleeping for {sleep_time} seconds...")
                    time.sleep(sleep_time)
            
            logger.info("="*60)
            logger.info("TRAIN ORDERS DATA LOADED SUCCESSFULLY")
            logger.info(f"Total orders loaded: {progress['total_orders_loaded']}")
            logger.info(f"Total order products loaded: {progress['total_order_products_loaded']}")
            logger.info("="*60)
            
        except KeyboardInterrupt:
            logger.warning("="*60)
            logger.warning("⚠️  INTERRUPTED BY USER")
            logger.warning("="*60)
            logger.info(f"Progress saved at order index: {progress['last_order_index']}")
            logger.info(f"Total orders loaded so far: {progress['total_orders_loaded']}")
            logger.info(f"Total order products loaded so far: {progress['total_order_products_loaded']}")
            logger.info(f"Remaining orders: {total_orders - current_index}")
            logger.info("You can safely re-run this cell to continue from where you left off.")
            logger.warning("="*60)
        
    except Exception as e:
        logger.error(f"Error loading train orders: {e}")
        conn.rollback()
        raise
    finally:
        conn.close()

# Execute train orders loading
load_train_orders_incrementally()

2025-12-19 22:36:32,336 - INFO - ============================================================
2025-12-19 22:36:32,337 - INFO - LOADING TRAIN ORDERS DATA (INCREMENTALLY)
2025-12-19 22:36:32,337 - INFO - ============================================================
2025-12-19 22:36:32,338 - INFO - Loading orders.csv...
2025-12-19 22:36:33,239 - INFO - Found 131209 train orders
2025-12-19 22:36:33,239 - INFO - Loading order_products__train.csv...
2025-12-19 22:36:33,390 - INFO - Found 1384617 train order products
2025-12-19 22:36:33,761 - INFO - Train batch completed: 61 orders, 699 order products | Progress: 61/131209 orders (0.0%)
2025-12-19 22:36:33,763 - INFO - Sleeping for 4 seconds...
2025-12-19 22:36:38,171 - INFO - Train batch completed: 93 orders, 885 order products | Progress: 154/131209 orders (0.1%)
2025-12-19 22:36:38,173 - INFO - Sleeping for 2 seconds...
2025-12-19 22:36:40,549 - INFO - Train batch completed: 113 orders, 1104 order products | Progress: 267/131209 orders (0.2

In [ ]:
# reset_progress()

2025-12-19 22:35:13,976 - INFO - Progress log reset. Next run will start from the beginning.
